# Species Tree reconstruction

In this example we will try to obtain 4 different species tree for phylome 15.

1. Duptree
2. Concatenation
3. Astral-PRO
4. SpeciesRax

First of all to import the functions do:

In [ ]:
import analyse_phylome as ap

In [ ]:
# set useful variable
duptreePath = ("/home/giacomo/master-thesis/Second_proj/obtain_phylome_data_scripts/duptree2")

outDir = "out_dir15"
treeFile = "test_data/best_trees15.txt"
duptreeFile = outDir + "/duptree_in.txt"
tagName = ""

ap.create_folder(outDir)

##### Duptree

Duptree is a super tree based method that has the advantage that it uses trees with duplications to infer the species tree that minimizes the number of inferred duplications according to reconciliation. It is one of the few programs you’ll find that returns a rooted tree. It will simpy need the best_trees file from PhylomeDB. 

obtain_duptree_file creates the input file for duptree. The gene-trees are simpy renamed with their species and you cand decide whether or not rooting them (midpoint=True)

In [ ]:
help(ap.obtain_duptree_file)

ap.obtain_duptree_file(treeFile, duptreeFile) #midpoint=True

help(ap.launch_duptree)

ap.launch_duptree(duptreeFile, "out_dir15/sp_duptree.nwk", duptreePath, outDir, tagName)


##### Concatenation tree

To do this, follow the tutorial in examples/example_cat.py and obtain the concat aln. Then, apply the phylomizer pipeline to this alignment. The alignment may be very long so it could be very computationally demanding.

The reconstruction pipeline is stored somewhere in /gpfs/projects/bsc40. -s t means that only the tree building step will be done.

`python reconstruction_pipeline.py -s t -i concat_aln -o out_tree`


#### Astral-PRO

"ASTRAL-Pro stands for ASTRAL for PaRalogs and Orthologs. ASTRAL is a tool for estimating an unrooted species tree given a set of unrooted gene trees and is statistically consistent under the multi-species coalescent model (and thus is useful for handling incomplete lineage sorting, i.e., ILS). ASTRAL-pro extends ASTRAL to allow multi-copy genes. ASTRAL-pro finds the species tree that has the maximum number of shared induced quartet tree equivalent classes with the set of gene trees, subject to the constraint that the set of bipartitions in the species tree comes from a predefined set of bipartitions." (see [here](https://github.com/chaoszhang/A-pro))

Though I suggest to use the C++ version of the tool which can be found [here](https://github.com/chaoszhang/ASTER/tree/548fd31493397ec44f5c8e3452c689e547b3e271) which is faster and can handle much larger jobs.

The function simply rename the trees with the species name as gene name.

In [ ]:
# simply rename the trees
ap.get_astral_pro_data(treeFile, "out_dir15/15_astral_trees.txt")

Once obtained the input trees to run astral-pro do:

`./astral_c/ASTER/astral-pro -o output_tree -u 1 -t threads best_trees_renamed.txt 2>&1 | tee apro.log`

Where -u stand for the level of verbosity. I suggest to redirect stdout to a logfile.

#### SpeciesRax

"SpeciesRax is part of the GeneRax tool and is available since GeneRax v2.0.0. SpeciesRax infers a rooted species tree from a set of unrooted gene trees." (see more [here](https://github.com/BenoitMorel/GeneRax)).

In order to run SpeciesRax need a family file where the tree file, the substition model and the mapping file between gene and species are explicited.

This function creates all of them. Note that to run SpeciesRax you don't need the alignment file.

In [ ]:
# speciesrax
ap.get_generax_data(treeFile, "out_dir15/sprax_15")

Then once obtained the data generax can be run in the cluster like this:

To load it

`module purge
module load gcc impi generax`


`mpiexec -np cores generax --families family.txt --strategy SKIP --si-strategy HYBRID --species-tree [MiniNJ or another species tre like duptree] --rec-model UndatedDTL --per-family-rates --prune-species-tree --si-estimate-bl --si-quartet-support --prefix sprax`